## IMD 0190 - TÓPICOS ESPECIAIS EM BUSINESS INTELIGENCE E ANALYTICS

#### Fernando Lucas, Renata Gurgel e Victor Gabriel

### Sumário
#### Principais pontos

- O portal de dados abertos expõe dados de transparência do TRE/RN agrupados em grandes áreas, tais como: Secretaria de Gestão de Pessoas; Secretaria de Tecnologia da Informação e Eleições; Zonas Eleitorais.

- O portal permite que os dados sejam baixados no formato csv sem necessidade de solicitação, basta acessar o site e selecionar o assunto de interesse. O site está disponível em: https://dados.tre-rn.jus.br/.

- O projeto sucintamente consiste na implementação de um agente de inteligência artificial para solucionar dúvidas e realizar análises iniciais sobre os dataframes.

- Utilização da API gratuita do *GEMINI* e *Ollama* como LLMs para desenvolvimento do projeto.

### Resumo

- Desenvolvimento de um Agente de IA que seja capaz de responder as questões dos usuários referente aos dados disponíveis no Portal de dados abertos do Tribunal Regional Eleitoral do Rio Grande do Norte (TRE/RN). O portal referido acima pode ser acessado através do link: https://dados.tre-rn.jus.br/group/pessoas.




### Importar bibliotecas

In [1]:
!pip install langchain_experimental langchain requests langchain-community langchain_google_genai google-generativeai langchain_ollama ollama

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.4 MB/s eta 0:00:00
  Using cached rsa-4.9-py3-none-any.whl.metadata (

### Zero-Shot Text Classification - An Real Application to Escola Judiciária Eleitoral do Rio Grande do Norte (EJE)

- This is a init for a reviews courses classifications using the method of Zero-Shot Text Classification

- We were based of: https://github.com/eliasjacob/deep_learning_gen_ai/blob/main/Notebook_10.ipynb

- We was incrementing the project step-by-step. So, some codes can be repetitive with different approaches

- The first implementation we have defined the schema using reviews segmented in *Positive*, *Negative* and *Neutral* generated by `DeepSeek-V3`. Then, use the `.with_structured_method` to format the output response about the sentiment.

#### Import Libraries

In [ ]:
pip install openpyxl pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 1.5 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


#### Defining the LLM models

In [ ]:
# Import ChatOllama class from langchain_ollama module
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
import os

gemini_api_key = ""
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=gemini_api_key,
    temperature=0.0
    )

# Initialize the ChatOllama model with specific parameters
model_llama = ChatOllama(
    model="llama3.1",  # Specify the model to use
    base_url="http://localhost:11434",  # Set the base URL where the Ollama service is running
    temperature=0.0,  # Set the temperature for response variability
)

/opt/disciplinas/gen-ai/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Lists of reviews

- We use *DeepSeek-V3* to generate feedbacks lists for each of the possible classifications: positive, negative and neutral;

- Then, for test purposes *unlabelled feedbacks* was set as the all reviews in each lists;


In [ ]:
# This is a example list of negative feedbacks
negative_feedbacks = [
    "O conteúdo é muito superficial, não aprofunda os temas.",
    "As aulas são longas e cansativas, difícil manter a atenção.",
    "Faltou material prático para aplicar o que foi ensinado.",
    "A plataforma é lenta e trava constantemente.",
    "O instrutor não explica de forma clara, fica confuso.",
    "O curso é caro para o pouco conteúdo que oferece.",
    "Não gostei da falta de interação com os instrutores.",
    "O certificado não é reconhecido no mercado.",
    "As atividades são repetitivas e pouco desafiadoras.",
    "O suporte é demorado e não resolve os problemas direito."
]
# This is a example list of positive feedbacks
positive_feedbacks = [
    "O curso é excelente, conteúdo muito bem organizado!",
    "Aprendi muito e consegui aplicar no meu trabalho. Recomendo!",
    "Instrutores claros e didáticos. Foi uma ótima experiência.",
    "O material de apoio é completo e fácil de entender.",
    "Adorei a plataforma, muito intuitiva e fácil de usar.",
    "O curso superou minhas expectativas. Parabéns à equipe!",
    "Conteúdo atualizado e relevante para o mercado atual.",
    "As atividades práticas ajudaram a fixar o conhecimento.",
    "Suporte rápido e eficiente. Tive todas as minhas dúvidas resolvidas.",
    "A interação com os alunos foi excelente."
]

# This is a example list of neutral feedbacks
neutral_feedbacks = [
    "O curso é bom, mas poderia ter mais exemplos práticos.",
    "O conteúdo é interessante, mas algumas aulas são muito longas.",
    "Gostei do material, mas a plataforma poderia ser mais estável.",
    "O curso atendeu às minhas expectativas, mas nada extraordinário.",
    "As explicações são claras, mas o ritmo é um pouco lento.",
    "O conteúdo é útil, mas senti falta de mais interação com os instrutores.",
    "O curso é razoável, mas o preço poderia ser mais acessível.",
    "Achei o material completo, mas algumas partes são repetitivas.",
    "O curso é bom para iniciantes, mas avançados podem achar básico.",
    "A experiência foi ok, mas esperava mais atividades práticas."
]

# This is a example list of feedbacks not classified
unlabelled_feedbacks = positive_feedbacks + negative_feedbacks + neutral_feedbacks

#### Method Implementation: '.with_structured_output()'

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal


class SentimentAnalysisResponse(BaseModel):
    """The response of a function that performs sentiment analysis on text."""

    # The sentiment label assigned to the text
    sentiment: Literal["positive", "neutral", "negative"] = Field(
        default_factory=str,
        description="The sentiment label assigned to the text. You can only have 'positive' or 'negative' as values.",
    )

#### Prompting the model

In [12]:
model_sentiment_gemini = gemini.with_structured_output(SentimentAnalysisResponse)

output = model_sentiment_gemini.invoke("A aula não foi muito boa e as atividades não foram interessantes como o esperado.")
output

SentimentAnalysisResponse(sentiment='negative')

- Conclusion: quickly and with little development effort, it is possible to obtain an application that solves a real problem for people who really need it.

#### Implementing the Agent

- Inicialmente criamos duas ferramentas, uma responsável por responder a avaliação analisada e a classificação, e a outra responsável por fazer a contabilização total de cada classe.

- Em contato com o cliente, percebemos que faz mais sentido a centralização em uma única ferramenta capaz de responder cada classificação por avaliação caso receba uma lista de comentários e a contabilização total, e caso receba apenas uma, responsa a classificação apenas.

- Os nomes delas são: `analyze_sentiment()` e `count_sentiments()`.

- Nós utilizamos a função do *LangChain* `create_tool_calling_agent` para criação do agente.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal, Union
from langchain.tools import tool
from langchain.memory import ConversationBufferMemory
from collections import Counter
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

class SentimentAnalysisResponse(BaseModel):
    """The response of a function that performs sentiment analysis on text."""

    # The sentiment label assigned to the text
    sentiment: Literal["positive", "neutral", "negative"] = Field(
        default_factory=str,
        description="The sentiment label assigned to the text. You can only have 'positive', 'neutral' or 'negative' as values.",
    )
model_sentiment_gemini = gemini.with_structured_output(SentimentAnalysisResponse)
# Criar um Tool para análise de sentimentos
@tool
def analyze_sentiment(texts: Union[str, List[str]]) -> dict:
    """
    Recebe um único texto ou uma lista de textos e retorna a classificação de sentimento de cada um.
    Se for uma lista, também retorna a contagem total de cada classificação.
    """

    if isinstance(texts, str):
        # Caso seja um único texto, retorna apenas sua classificação
        return model_sentiment_gemini.invoke(texts)

    elif isinstance(texts, list):
        # Caso seja uma lista, processa cada um e gera um resumo da contagem
        sentiment_counts = Counter()
        individual_results = []

        for text in texts:
            result = model_sentiment_gemini.invoke(text)
            sentiment_counts[result.sentiment] += 1
            individual_results.append(result)

        return {
            "individual_results": individual_results,
            "total_counts": dict(sentiment_counts)
        }


# Criar um Tool para contar os sentimentos
@tool
def count_sentiments(texts: List[str]) -> dict:
    """Recebe uma lista de textos e retorna a contagem de sentimentos (positivo, neutro, negativo)."""
    sentiment_counts = Counter()

    for text in texts:
        output = model_sentiment_gemini.invoke(text)
        sentiment_counts[output.sentiment] += 1

    return dict(sentiment_counts)

# Criar um agente com memória
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define a prompt template for the chat, including system instructions and placeholders
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
tools=[analyze_sentiment]
# Inicializar o agente
agent = create_tool_calling_agent(
    tools=tools,  # Tools disponíveis para o agente
    llm=gemini,
    prompt=prompt
)

/tmp/ipykernel_76519/1241167377.py:59: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


- A fonte de dados é proveniente do google forms e após todos os alunos responderem, os dados podem ser baixados em diversos formatos, dentre eles: **csv**.

- Dessa forma, importamos o dataframe e convertemos para uma lista de com as avaliações não nulas.

In [ ]:
import pandas as pd

# Configuração da fonte de dados
sheet_id = ""
sheet_name = ""  # Certifique-se de usar o nome correto da aba

# URL gerada pelo Google Sheets para exportação como CSV
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Carregar os dados no Pandas
df_drive = pd.read_csv(url, header=None, usecols=[14], dtype=str)
df_manual = pd.read_csv("./Avaliação de Reação.csv", usecols=[14])

reviews = df_manual.dropna()
reviews.head(50)
reviews_filtered = reviews.head()
unlabelled_feedbacks = reviews_filtered['Unnamed: 14'].tolist()
print(len(unlabelled_feedbacks))


5


- Após definição do agente, podemos utilizar o `AgentExecutor()` também do *LangChain* para executar o agente recebendo uma entrada do usuário.

- Utilizamos 5 dados não classificados previamente obtidos pelo dataframe inserido anteriormente.

In [10]:
# Create an instance of AgentExecutor with specified agent, tools, and verbosity
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Invoke the agent_executor with a dictionary containing the input query
agent_executor.invoke(
    {
        "input": f"Informe a classificação para cada um dos textos em {unlabelled_feedbacks}",  # The input query asking for the price of Bitcoin
    }
)



> Entering new AgentExecutor chain...

Invoking: `analyze_sentiment` with `{'texts': ['Tudo ótimo.', 'Ótimo curso.', 'Treinamento oportuno', 'Nada a acrescentar', 'Curso ótimo!']}`


{'individual_results': [SentimentAnalysisResponse(sentiment='positive'), SentimentAnalysisResponse(sentiment='positive'), SentimentAnalysisResponse(sentiment='positive'), SentimentAnalysisResponse(sentiment=''), SentimentAnalysisResponse(sentiment='positive')], 'total_counts': {'positive': 4, '': 1}}A classificação de sentimento para os textos fornecidos é:
- Tudo ótimo.: positivo
- Ótimo curso.: positivo
- Treinamento oportuno: positivo
- Nada a acrescentar: sem sentimento
- Curso ótimo!: positivo

Além disso, a contagem total de cada classificação é:
- positivo: 4
- sem sentimento: 1

> Finished chain.


{'input': "Informe a classificação para cada um dos textos em ['Tudo ótimo.', 'Ótimo curso.', 'Treinamento oportuno', 'Nada a acrescentar', 'Curso ótimo!']",
 'output': 'A classificação de sentimento para os textos fornecidos é:\n- Tudo ótimo.: positivo\n- Ótimo curso.: positivo\n- Treinamento oportuno: positivo\n- Nada a acrescentar: sem sentimento\n- Curso ótimo!: positivo\n\nAlém disso, a contagem total de cada classificação é:\n- positivo: 4\n- sem sentimento: 1'}

- Criamos uma nova ferramenta capaz de receber o dataframe, fazer o tratamento para converter em lista e fazer a classificação e contabilização de cada classe.

- Como o setor que utilizará a ferramenta tem um padrao na fonte de dados em que a coluna que interessa é sempre a mesma e, devido a ela não ser nomeada (padrão pós google forms), utilizamos a ferramenta para receber apenas o dataframe sem identificação da coluna alvo.

In [ ]:
@tool
def process_dataframe_for_sentiment(df: pd.DataFrame) -> dict:
    """
    Recebe um DataFrame e sempre usa a coluna de índice 14 para análise de sentimentos.
    Retorna os resultados individuais e a contagem total de classificações.
    """

    # Verifica se o índice 14 está dentro do DataFrame
    if len(df.columns) <= 14:
        return {"error": f"O DataFrame possui apenas {len(df.columns)} colunas. A coluna de índice 14 não existe."}

    column = df.columns[14]  # Obtém o nome da coluna de índice 14
    texts = df[column].dropna().tolist()

    if not texts:
        return {"error": "A coluna 14 não contém valores válidos para análise."}

    # Chamar analyze_sentiment para cada texto
    sentiment_counts = Counter()
    individual_results = []

    for text in texts:
        result = model_sentiment_gemini.invoke(text)
        sentiment_counts[result.sentiment] += 1
        individual_results.append(result)

    return {
        "individual_results": individual_results,  # Lista de SentimentAnalysisResponse
        "total_counts": dict(sentiment_counts)  # Contagem dos sentimentos
    }

- Implementação da interface no streamlit

In [ ]:
import streamlit as st
import matplotlib.pyplot as plt
# 🔹 Interface do Streamlit
st.title("Análise de Sentimentos com LLM 🚀")
st.write("Digite um texto, uma lista de textos ou faça upload de um CSV.")

# 📌 **Entrada de texto do usuário**
user_input = st.text_area("Digite um comando para o agente:")

# 📌 **Upload do CSV**
uploaded_file = st.file_uploader("Ou carregue um arquivo CSV", type=["csv"])

df = None
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.subheader("📊 Prévia do DataFrame")
    st.write(df.head())

if st.button("Executar Análise"):
    if user_input:
        with st.spinner("O agente está processando sua solicitação..."):
            result = agent_executor.invoke({"input": user_input})
    elif df is not None:
        with st.spinner("O agente está analisando os dados..."):
            result = agent_executor.invoke({"input": "Analise o sentimento do DataFrame."})

    if "error" in result:
        st.error(result["error"])
    else:
        if "individual_results" in result:
            st.subheader("📌 Classificação Individual")
            st.write(pd.DataFrame(result["individual_results"]))

        if "total_counts" in result:
            st.subheader("📊 Resumo da Análise")
            st.write(result["total_counts"])

            # Criando o gráfico de barras
            st.subheader("📊 Distribuição dos Sentimentos")
            fig, ax = plt.subplots()
            ax.bar(result["total_counts"].keys(), result["total_counts"].values(), color=["green", "gray", "red"])
            ax.set_xlabel("Sentimentos")
            ax.set_ylabel("Quantidade")
            ax.set_title("Distribuição dos Sentimentos no Dataset")
            st.pyplot(fig)

### Pandas Agent

In [ ]:
# Import the function to create an agent that works with pandas DataFrames
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
# Import the pandas library for data manipulation
import pandas as pd

# Read the Servidor_Ativo dataset from the provided URL into a pandas DataFrame
#df = pd.read_csv("/content/servidor_ativo.csv", delimiter=";")
# Read the Ex_Servidor dataset from the provided URL into a pandas DataFrame
#df1 = pd.read_csv("/content/ex_servidor.csv", delimiter=";")
# Read the Servidor_Inativo dataset from the provided URL into a pandas DataFrame
#df2 = pd.read_csv("/content/servidor_inativo.csv", delimiter=";")
# Read the Servidor_Ausente dataset from the provided URL into a pandas DataFrame
#df3 = pd.read_csv("/content/servidor_ausente.csv", delimiter=";")
# Read the Servidor_Instituidor_Pensao dataset from the provided URL into a pandas DataFrame
#df4 = pd.read_csv("/content/servidor_instituidor_pensao.csv", delimiter=";")
df_diarias = pd.read_csv("./diarias-historico.csv", delimiter=";")
df_passagens = pd.read_csv("./passagens-historico.csv", delimiter=";")

# Display the first few rows of the DataFrame to verify the data has been loaded correctly
df_diarias.head()

In [ ]:
df_passagens.head()

In [ ]:
df_passagens['COMPANHIA'].nunique()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

import os

gemini_api_key = ""
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=gemini_api_key,
    temperature=0.0
    )

In [ ]:
# Criar um PromptTemplate para reformular as respostas de forma mais conversacional
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="""
    Você é um assistente amigável e prestativo. Sua tarefa é reformular a resposta fornecida de maneira clara, explicativa e simpática. 
    A resposta deve ser dada em Português-Brasil e deve soar natural, como se fosse uma conversa.

    Resposta original: {query}

    Resposta reformulada:
    """
)
pipeline = prompt_template | gemini

In [ ]:
# Adicionar um prefixo ao prompt para explicar os dataframes
prefixo = """
Você tem acesso a dois dataframes:
1. `df_diarias`: Contém informações sobre diárias pagas.
2. `df_passagens`: Contém informações sobre passagens aéreas.

Use o dataframe apropriado para responder às perguntas.
"""
# Create a pandas dataframe agent using the specified LLM model and dataframe
pandas_agent = create_pandas_dataframe_agent(
    llm=gemini,  # Pass the initialized ChatOpenAI model
    df=[df_diarias, df_passagens],  # Provide the dataframe to be used by the agent
    verbose=True,  # Enable verbose mode for detailed logging
    agent_type="zero-shot-react-description",  # Specify the type of agent to create
    allow_dangerous_code=True,  # Opt-in to allow the use of the REPL tool
    prefix=prefixo
)

In [ ]:
def humanized_agent_response(query):
    try:
        # Obter a resposta original do pandas_agent
        resposta_original = pandas_agent.invoke(query)

        # Garantir que a resposta seja uma string
        if isinstance(resposta_original, dict):
            resposta_original = resposta_original.get("output", "") or resposta_original.get("input", "")
        resposta_original = str(resposta_original).strip()

        # Passar a resposta original para o pipeline para humanização
        resposta_humanizada = pipeline.invoke({"query": resposta_original})

        # Garantir que a resposta reformulada seja uma string
        if isinstance(resposta_humanizada, dict):
            resposta_humanizada = resposta_humanizada.get("output", "") or resposta_humanizada.get("query", "")
        resposta_humanizada = str(resposta_humanizada.content).strip()

        return resposta_humanizada

    except Exception as e:
        return f"Desculpe, ocorreu um erro ao reformular a resposta: {str(e)}"

In [ ]:
print(df_diarias['SERVIDOR'].nunique())

In [ ]:
query = "Pode me informar a quantidade de servidores diferentes?"
response = pandas_agent.invoke(query)
print(response)

#### Streamlit

### RAG

In [ ]:
!pip install langchain faiss-cpu

In [ ]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(file_path="/content/servidor_ativo.csv", csv_args={"delimiter": ";"})
documentos = loader.load()

In [ ]:
def reformat_document(doc):
    # Divide o conteúdo em linhas
    linhas = doc.page_content.split("\n")
    info = {}
    # Processa cada linha, extraindo chave e valor
    for linha in linhas:
        if ":" in linha:
            chave, valor = linha.split(":", 1)
            # Se o valor estiver vazio após remover espaços, atribua "não informado"
            valor_formatado = valor.strip() if valor.strip() else "não informado"
            info[chave.strip()] = valor_formatado
    # Cria um texto formatado de forma natural
    texto_formatado = (
        f"{info.get('nome', 'não informado')}, matrícula {info.get('matricula', 'não informado')}, "
        f"lotado em {info.get('lotacao', 'não informado')}, nível {info.get('nivel', 'não informado')}, "
        f"cargo {info.get('cargo', 'não informado')}, área {info.get('area', 'não informado')}, "
        f"situação {info.get('situacao', 'não informado')}, especialista em {info.get('especialidade', 'não informado')} ingresso em {info.get('ingresso', 'não informado')}."
    )
    # Atualiza o conteúdo do documento
    #doc.page_content = texto_formatado
    return texto_formatado

# Aplica a reformatação a todos os documentos
#documentos_formatados = [reformat_document(doc) for doc in documentos]
documento = [reformat_document(doc) for doc in documentos]

In [ ]:
print(documento[:1000])

In [ ]:
# Definir o número de linhas por chunk
num_linhas_por_chunk = 5

# Dividir o documento único em linhas
linhas = documento.split("\n")

# Criar chunks de 50 linhas cada
chunks = ["\n".join(linhas[i : i + num_linhas_por_chunk]) for i in range(0, len(linhas), num_linhas_por_chunk)]

# Exibir informações sobre os chunks gerados
print(f"Total de chunks gerados: {len(chunks)}")
print("\n--- Exemplo de um Chunk ---\n")
print(chunks[0])  # Exibir o primeiro chunk para conferência


In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here")

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-001")

In [ ]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
documentos = [Document(page_content=chunk) for chunk in chunks]
vectorstore = FAISS.from_documents(documentos, embeddings)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
chain = load_qa_chain(gemini, chain_type="stuff")
qa_chain = RetrievalQA(combine_documents_chain=chain, retriever=retriever)

In [ ]:
retrieved_docs = retriever.invoke("Quantos servidores são especialistas em Operação de Computadores?")
for doc in retrieved_docs:
    print(doc.page_content)

In [ ]:
pergunta = "Quantos servidores são especialistas em Operação de Computadores?"
resposta = qa_chain.run(pergunta)
print(resposta)